In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, recall_score
from sklearn.decomposition import PCA
from scipy.stats import ttest_ind
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Ignorar warnings de convergência
import warnings

warnings.filterwarnings("ignore")




In [2]:
df = pd.read_csv("dados\dados_tratados.csv")

In [3]:
df.columns.tolist()

['idade',
 'renda_anual',
 'status_moradia_atual',
 'tempo_emprego',
 'proposito_emprestimo',
 'risco_cliente',
 'valor_total_emprestimo',
 'taxa_juros',
 'proporcao_emprestimo_renda',
 'negativado',
 'tempo_relacionamento_bancario',
 'status_emprestimo',
 'razao_tempo_emprego_banco',
 'risco_alto_binario',
 'taxa_comprometimento_alta',
 'razao_tempo_banco_idade',
 'faixa_taxa_juros',
 'faixa_etaria',
 'taxa_juros_alta']

In [4]:
def clf_metrics_com_return(modelo, X, y_true, label, plot_conf_mat=True, print_cr=True):
    
    if print_cr:
        print(f"\nMétricas de avaliação de {label}:\n")
    
    y_pred = modelo.predict(X)

    if plot_conf_mat:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))

        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax[0]) 
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize="all", ax=ax[1])
        plt.show()

    if print_cr:
        print(classification_report(y_true, y_pred))
    
    return classification_report(y_true, y_pred, output_dict=True)

In [5]:
# class OutlierDetector(BaseEstimator, TransformerMixin):
#     def __init__(self, col_categoria, col_valor, faixa_de_tolerancia=1.5):
#         self.col_categoria = col_categoria
#         self.col_valor = col_valor
#         self.faixa_de_tolerancia = faixa_de_tolerancia
#         self.limites_outliers = {}

#     def fit(self, X, y=None):
#         # Calcula os limites de outliers (IQR) para cada categoria do propósito
#         for categoria in X[self.col_categoria].unique():
#             subset = X[X[self.col_categoria] == categoria][self.col_valor]
#             Q1 = subset.quantile(0.25)
#             Q3 = subset.quantile(0.75)
#             IQR = Q3 - Q1
#             limite_inferior = Q1 - self.faixa_de_tolerancia * IQR
#             limite_superior = Q3 + self.faixa_de_tolerancia * IQR
#             self.limites_outliers[categoria] = (limite_inferior, limite_superior)
#         return self

#     def transform(self, X, y=None):
#         # Cria uma coluna para marcar os outliers como 1 ou 0
#         X = X.copy()
#         X['outlier_valor_proposito'] = X.apply(
#             lambda row: 1 if (row[self.col_valor] < self.limites_outliers[row[self.col_categoria]][0] or
#                               row[self.col_valor] > self.limites_outliers[row[self.col_categoria]][1])
#             else 0, axis=1
#         )
#         return X

In [6]:
colunas_categoricas = ['proposito_emprestimo', 'status_moradia_atual', 'faixa_etaria']
colunas_numericas = df.drop(columns=['status_emprestimo', "negativado", 'risco_alto_binario',
 'taxa_comprometimento_alta',
 'faixa_taxa_juros',
 'faixa_etaria',
 'taxa_juros_alta']).select_dtypes(include='number').columns.tolist()
# Tirei negativado pq ele também é binário (0, 1)


# Configurando o OneHotEncoder no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop=None, sparse_output=False, handle_unknown='ignore'), colunas_categoricas),
        ('num', StandardScaler(), colunas_numericas)
    ],
    remainder='passthrough'  # Mantém as outras colunas sem mudanças
)

In [7]:
X = df.drop(columns=['status_emprestimo'], axis=1)
y = df['status_emprestimo']

X_teste, X_treino, y_teste, y_treino = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y, shuffle=True)

In [8]:
# Vou fazer alguns testes aqui. Tanto implementar balanceamento dentro dos hiperparâmetros dos estimadores, como SMOTE



# Regressão logística
# pipe_logit_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
#                         ('preprocessor', preprocessor),
#                        ("logit", LogisticRegression(random_state=42))])

pipe_logit_balanceado = Pipeline([('preprocessor', preprocessor),
                       ("logit", LogisticRegression(random_state=42, class_weight='balanced', max_iter=500, n_jobs=-1))])

pipe_logit_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
                       ("logit", LogisticRegression(random_state=42, max_iter=500, n_jobs=-1))])


# Random Forest
# pipe_rf_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
#                         ('preprocessor', preprocessor),
#                     ("rf", RandomForestClassifier(random_state=42))])

pipe_rf_balanceado = Pipeline([('preprocessor', preprocessor),
                    ("rf", RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1))])

pipe_rf_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
                    ("rf", RandomForestClassifier(random_state=42, n_jobs=-1))])

# Decision Tree
# pipe_dt_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
#                         ('preprocessor', preprocessor),
#                     ("dt", DecisionTreeClassifier(random_state=42))])

pipe_dt_balanceado = Pipeline([('preprocessor', preprocessor),
                    ("dt", DecisionTreeClassifier(random_state=42, class_weight='balanced'))])

pipe_dt_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
                    ("dt", DecisionTreeClassifier(random_state=42))])



# SVM

# pipe_svm_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
#                          ('preprocessor', preprocessor),
#                      ("svm", SVC(random_state=42))])

pipe_svm_balanceado = Pipeline([('preprocessor', preprocessor),
                     ("svm", SVC(random_state=42, class_weight='balanced'))])

pipe_svm_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
                     ("svm", SVC(random_state=42))])


# Catboost 

# pipe_catboost_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
#                               ('preprocessor', preprocessor),
#                               ("catboost", CatBoostClassifier(random_state=42, verbose=0))])

pipe_catboost_balanceado = Pipeline([('preprocessor', preprocessor),
                                     ("catboost", CatBoostClassifier(random_state=42, auto_class_weights='Balanced', verbose=0))])

# CatBoost com SMOTE
pipe_catboost_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
                                   ("catboost", CatBoostClassifier(random_state=42, verbose=0))])

# ================================================


dict_pipes = {"logit balanceado": pipe_logit_balanceado,
              "random forest balanceado": pipe_rf_balanceado,
              "decision tree balanceado": pipe_dt_balanceado,
              "svm balanceado": pipe_svm_balanceado,
              "catboost balancado": pipe_catboost_balanceado}

dict_pipes_smote = {"logit smote": pipe_logit_smote,
                    "random forest smote": pipe_rf_smote,
                    "decision tree smote": pipe_dt_smote,
                    "svm smote": pipe_svm_smote,
                    "catboost smote": pipe_catboost_smote}

In [9]:
# # Mesma coisa usando CV

# # Configuração do StratifiedKFold
# kf = StratifiedKFold(n_splits=5)

# # Inicialização do dicionário de resultados
# resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

# # Loop pelos pipelines
# for nome_modelo, pipeline in dict_pipes.items():
#     recall_treino_lista = []
#     recall_teste_lista = []
    
#     # Realiza a validação cruzada estratificada
#     for indice_treino, indice_valida in kf.split(X_treino, y_treino):
#         X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
#         y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]
        
#         # Treina o modelo
#         pipeline.fit(X_treino_split, y_treino_split)
        
#         # Avalia o modelo no treino
#         y_pred_treino = pipeline.predict(X_treino_split)
#         recall_treino = recall_score(y_treino_split, y_pred_treino, average='binary')
#         recall_treino_lista.append(recall_treino)
        
#         # Avalia o modelo no teste (validação)
#         y_pred_valida = pipeline.predict(X_valida_split)
#         recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
#         recall_teste_lista.append(recall_teste)
    
#     # Calcula a média dos recalls
#     recall_treino_medio = np.mean(recall_treino_lista)
#     recall_teste_medio = np.mean(recall_teste_lista)
    
#     # Armazena os resultados
#     resultado_experimentos["estimador"].append(nome_modelo)
#     resultado_experimentos["recall_treino"].append(recall_treino_medio)
#     resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
#     print(f'Treinamento do modelo {nome_modelo} finalizado')

# # Resultado final em DataFrame
# df_resultados = pd.DataFrame(resultado_experimentos)

# # Em casos de underfit, calcula a diferença entre o recall de treino e teste
# df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# # Ordena os resultados pelo recall no teste
# df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

# df_resultados

In [10]:
# transformar idade em uma coluna faixa etária e dividir em bins
# Deixar para transformar no one hot enconder só depois pq enquanto ainda for faixa vai ser utilizado
# para pegar outlier de renda ou valor total do empréstimo

In [12]:
# Mesma coisa usando CV e SMOTE

# Configuração do StratifiedKFold
kf = StratifiedKFold(n_splits=5)

# Inicialização do dicionário de resultados
resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

smote = SMOTE(random_state=42, n_jobs=-1)

X = df.drop(columns=['status_emprestimo'], axis=1)
y = df['status_emprestimo']

X_teste, X_treino, y_teste, y_treino = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y, shuffle=True)


# Loop pelos pipelines
for nome_modelo, pipeline in dict_pipes_smote.items():
    recall_treino_lista = []
    recall_teste_lista = []
    
    # Realiza a validação cruzada estratificada
    for indice_treino, indice_valida in kf.split(X_treino, y_treino):
        X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
        y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]


        # Aplica a transformação nas variáveis categóricas e numéricas
        X_treino_split_transformado = preprocessor.fit_transform(X_treino_split)
        X_valida_split_transformado = preprocessor.transform(X_valida_split)  # Não aplicamos fit no conjunto de validação

        # Aplica o SMOTE somente no conjunto de treino
        X_treino_smote, y_treino_smote = smote.fit_resample(X_treino_split_transformado, y_treino_split)

        # Treina o modelo
        pipeline.fit(X_treino_smote, y_treino_smote)

        
        # Avalia o modelo no treino
        y_pred_treino = pipeline.predict(X_treino_smote)
        recall_treino = recall_score(y_treino_smote, y_pred_treino, average='binary')
        recall_treino_lista.append(recall_treino)
        
        # Avalia o modelo no teste (validação)
        y_pred_valida = pipeline.predict(X_valida_split_transformado)
        recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
        recall_teste_lista.append(recall_teste)
    
    # Calcula a média dos recalls
    recall_treino_medio = np.mean(recall_treino_lista)
    recall_teste_medio = np.mean(recall_teste_lista)
    
    # Armazena os resultados
    resultado_experimentos["estimador"].append(nome_modelo)
    resultado_experimentos["recall_treino"].append(recall_treino_medio)
    resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
    print(f'Treinamento do modelo {nome_modelo} finalizado')



# Resultado final em DataFrame
df_resultados = pd.DataFrame(resultado_experimentos)

# Em casos de underfit, calcula a diferença entre o recall de treino e teste
df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# Ordena os resultados pelo recall no teste
df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

df_resultados

Treinamento do modelo logit smote finalizado
Treinamento do modelo random forest smote finalizado
Treinamento do modelo decision tree smote finalizado
Treinamento do modelo svm smote finalizado
Treinamento do modelo catboost smote finalizado


,estimador,recall_treino,recall_teste,gap
0,logit smote,0.819501,0.796088,0.023413
3,svm smote,0.812827,0.767237,0.045590
4,catboost smote,0.966140,0.717359,0.248780
2,decision tree smote,1.000000,0.709535,0.290465
1,random forest smote,1.000000,0.704645,0.295355


,idade,renda_anual,status_moradia_atual,tempo_emprego,proposito_emprestimo,risco_cliente,valor_total_emprestimo,taxa_juros,proporcao_emprestimo_renda,negativado,tempo_relacionamento_bancario,razao_tempo_emprego_banco,risco_alto_binario,taxa_comprometimento_alta,razao_tempo_banco_idade,faixa_taxa_juros,faixa_etaria,taxa_juros_alta
0,37,35000,aluguel,0.0,educacao,2,6000,11.49,0.17,0,14,0.000,0,0,0.378378,4,faixa_etaria_35_38,0
1,22,56000,propria,6.0,saude,3,4000,13.35,0.07,0,2,3.000,0,0,0.090909,5,faixa_etaria_20_23,0
2,29,28800,propria,8.0,pessoal,1,6000,8.90,0.21,0,10,0.800,0,0,0.344828,3,faixa_etaria_29_32,0
3,30,70000,aluguel,14.0,empreendimento,2,12000,11.11,0.17,0,5,2.800,0,0,0.166667,4,faixa_etaria_29_32,0
4,22,60000,aluguel,2.0,saude,1,6000,6.92,0.10,0,3,0.667,0,0,0.136364,2,faixa_etaria_20_23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57023,34,120000,hipoteca,5.0,educacao,4,25000,15.95,0.21,1,10,0.500,1,0,0.294118,6,faixa_etaria_32_35,1
57024,28,28800,aluguel,0.0,saude,3,10000,12.73,0.35,0,8,0.000,0,1,0.285714,5,faixa_etaria_26_29,0
57025,23,44000,aluguel,7.0,educacao,4,6800,16.00,0.15,0,2,3.500,1,0,0.086957,7,faixa_etaria_23_26,1
57026,22,30000,aluguel,2.0,educacao,1,5000,8.90,0.17,0,3,0.667,0,0,0.136364,3,faixa_etaria_20_23,0
